# 全ての観光地情報を取得

In [1]:
from selenium import webdriver
browser = webdriver.Chrome()
browser.get('https://scraping-for-beginner.herokuapp.com/ranking/')

In [5]:
from selenium.webdriver.common.by import By

## 観光地名一覧

In [6]:
elems_rankingBox = browser.find_elements(By.CLASS_NAME, 'u_areaListRankingBox')

In [7]:
len(elems_rankingBox)

10

In [8]:
# 全ての観光地名を格納するリスト titlesを用意
titles = []

for elem_rankingBox in elems_rankingBox:
    elem_title = elem_rankingBox.find_element(By.CLASS_NAME, 'u_title')
    title = elem_title.text.split('\n')[1]
    titles.append(title)

In [9]:
titles

['観光地 1',
 '観光地 2',
 '観光地 3',
 '観光地 4',
 '観光地 5',
 '観光地 6',
 '観光地 7',
 '観光地 8',
 '観光地 9',
 '観光地 10']

## 総合評価一覧

In [10]:
# やり方１
# totalEvaluates = []

# for elem_rankingBox in elems_rankingBox:
#     elem_evaluate = elem_rankingBox.find_element(By.CLASS_NAME, 'u_rankBox').find_element(By.CLASS_NAME, 'evaluateNumber')
#     totalEvaluates.append(elem_evaluate.text)
# totalEvaluates

In [11]:
# やり方2(手順を確認しながらfor文にしていくステップ方式)
elems_rank = browser.find_elements(By.CLASS_NAME, 'u_rankBox')
len(elems_rank)

10

In [12]:
elems_rank[0].find_element(By.CLASS_NAME, 'evaluateNumber').text

'4.7'

In [13]:
elems_rank = browser.find_elements(By.CLASS_NAME, 'u_rankBox')

ranks = []
for elem_rank in elems_rank:
    elem_rank = elem_rank.find_element(By.CLASS_NAME, 'evaluateNumber')
    rank = float(elem_rank.text) # Webスクレイピングで取ってきた数値は基本的に文字列になっている
    ranks.append(rank)

In [14]:
ranks

[4.7, 4.7, 4.6, 4.5, 4.5, 4.4, 4.3, 4.3, 4.2, 4.1]

## 各カテゴリ評価一覧
### リストのリスト(二次元の入れ子構造)にする

In [15]:
elems = browser.find_elements(By.CLASS_NAME, 'u_categoryTipsItem')
len(elems)

10

In [16]:
elem = elems[0]
elems_rank = elem.find_elements(By.CLASS_NAME, 'is_rank')

In [17]:
len(elems_rank)

4

In [18]:
# 二重にネストしたループを回してカテゴリ別評価数を格納する
categories = []

for elem in elems:
    elems_rank = elem.find_elements(By.CLASS_NAME, 'is_rank')

    _ranks = [] # 一時的な保存用

    for elem_rank in elems_rank:
        rank = elem_rank.find_element(By.CLASS_NAME, 'evaluateNumber').text
        _ranks.append(float(rank))
        
    categories.append(_ranks)

In [89]:
categories

[[4.6, 4.5, 4.9, 4.2],
 [4.6, 4.5, 4.9, 4.2],
 [4.5, 4.4, 4.8, 4.1],
 [4.4, 4.4, 4.8, 4.0],
 [4.4, 4.3, 4.7, 4.0],
 [4.3, 4.3, 4.7, 3.9],
 [4.2, 4.2, 4.6, 3.8],
 [4.2, 4.2, 4.6, 3.8],
 [4.1, 4.1, 4.5, 3.7],
 [4.0, 4.1, 4.4, 3.6]]

### Pandasの形式に変換

In [90]:
import pandas as pd

In [91]:
df = pd.DataFrame()

In [92]:
df['観光地名'] = titles
df['総合評価'] = ranks

In [93]:
df

,観光地名,総合評価
0,観光地 1,4.7
1,観光地 2,4.7
2,観光地 3,4.6
3,観光地 4,4.5
4,観光地 5,4.5
5,観光地 6,4.4
6,観光地 7,4.3
7,観光地 8,4.3
8,観光地 9,4.2
9,観光地 10,4.1


In [94]:
# 一旦カテゴリ別評価だけ別枠で作る
df_categories = pd.DataFrame(categories)
# 列の名前を自動で付く0, 1, 2, 3から変更する
df_categories.columns = ['楽しさ', '人混みの多さ', '景色', 'アクセス']
df_categories.rows = ['壱', '弐', '参', '']
df_categories

,楽しさ,人混みの多さ,景色,アクセス
0,4.6,4.5,4.9,4.2
1,4.6,4.5,4.9,4.2
2,4.5,4.4,4.8,4.1
3,4.4,4.4,4.8,4.0
4,4.4,4.3,4.7,4.0
5,4.3,4.3,4.7,3.9
6,4.2,4.2,4.6,3.8
7,4.2,4.2,4.6,3.8
8,4.1,4.1,4.5,3.7
9,4.0,4.1,4.4,3.6


In [ ]:
# ２つの表を横並びで連結する
pd.concat([df, df_categories])